In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (confusion_matrix, classification_report,  roc_auc_score, 
accuracy_score, precision_score, recall_score, f1_score, auc, precision_recall_curve, average_precision_score)
from sklearn.ensemble import (RandomForestClassifier)
from sklearn.svm import SVC

In [2]:
pd.set_option('display.max_rows', 400)
pd.set_option('max_colwidth', 120)
pd.set_option('display.max_columns', None)

In [3]:
train = pd.read_pickle('./feature/feat_v1.pkl')
train

,word_count,pub_time,abstract_len,headline_len,len_lead,n_comment,is_popular,is_noon,is_weekday,group_by_news_desk,feature_news_desk,group_by_section_name,feature_section_name,group_by_subsection_name,feature_subsection_name,group_by_type_of_material,feature_type_of_material,is_party,is_congress,is_president,is_spell,is_crossword,is_game,sentiment_tb_polar_abs,sentiment_tb_subj,sentiment_vader_pos,sentiment_vader_compound,sentiment_snow_pos,sentiment_abstract_tb_polar,sentiment_abstract_tb_polar_abs,sentiment_abstract_tb_subj,sentiment_abstract_tb_pos,sentiment_abstract_tb_neg,sentiment_abstract_vader_neu,sentiment_abstract_vader_pos,sentiment_abstract_vader_compound,sentiment_abstract_snow,sentiment_abstract_snow_abs,sentiment_abstract_snow_pos,sentiment_abstract_snow_neg
54115,1088,18,28,10,33,0,0,0,1,0.200885,0.2,0.198759,0.2,0.206438,0.2,0.566265,0.6,0,0,0,0,0,0,0.150000,0.250000,0.478,0.7506,1,0.225000,0.225000,0.400000,1,0,0.680,0.173,0.2298,0.534707,0.069414,1,0
16691,1286,8,23,11,35,12,1,1,1,0.200885,0.2,0.198759,0.2,0.209779,0.2,0.566265,0.6,0,0,0,0,0,0,0.211039,0.495130,0.000,0.0000,0,-0.200000,0.200000,0.400000,0,1,0.887,0.113,0.4215,0.220917,0.558166,0,1
12519,1502,22,16,8,32,0,0,0,1,0.265251,0.3,0.216640,0.3,0.266902,0.3,0.566265,0.6,0,0,1,0,0,0,0.000000,0.000000,0.182,-0.3182,0,-0.250000,0.250000,0.312500,0,1,0.722,0.000,-0.6597,0.074892,0.850216,0,1
9076,1581,9,17,12,33,219,1,1,1,0.265251,0.3,0.216640,0.3,0.266902,0.3,0.566265,0.6,0,0,1,0,0,0,0.000000,0.000000,0.000,-0.4939,0,-0.050000,0.050000,0.400000,0,1,0.711,0.289,0.6249,0.459658,0.080683,0,1
23387,1296,7,33,12,73,85,1,1,1,0.236273,0.2,0.229108,0.2,NaN,NaN,0.566265,0.6,0,0,0,0,0,0,0.600000,1.000000,0.000,-0.6249,0,0.383333,0.383333,0.544444,1,0,0.771,0.176,0.6808,0.032445,0.935111,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44436,300,4,10,4,35,0,0,0,1,0.000000,0.0,0.000000,0.0,NaN,NaN,0.000000,0.0,0,0,0,0,0,0,0.000000,0.000000,0.000,0.0000,0,0.000000,0.000000,0.000000,0,1,1.000,0.000,0.0000,0.279163,0.441675,0,1
44435,48,4,9,11,24,0,0,0,1,0.176948,0.2,0.002232,0.2,NaN,NaN,0.000000,0.0,0,0,0,0,0,0,0.000000,0.000000,0.000,-0.5267,0,0.000000,0.000000,0.000000,0,1,1.000,0.000,0.0000,0.300779,0.398442,0,1
40486,49,4,9,12,35,0,0,0,1,0.176948,0.2,0.002232,0.2,NaN,NaN,0.000000,0.0,0,0,0,0,0,0,0.078788,0.307071,0.191,0.3818,0,0.000000,0.000000,0.000000,0,1,1.000,0.000,0.0000,0.423108,0.153783,0,1
29147,47,6,9,13,20,0,0,1,1,0.176948,0.2,0.002232,0.2,NaN,NaN,0.000000,0.0,0,0,0,0,0,0,0.000000,0.000000,0.000,-0.7650,0,0.000000,0.000000,0.000000,0,1,1.000,0.000,0.0000,0.232313,0.535374,0,1


In [4]:
X = train.drop(columns=['is_popular','n_comment','group_by_subsection_name','feature_subsection_name'])
y = train['is_popular']
X

,word_count,pub_time,abstract_len,headline_len,len_lead,is_noon,is_weekday,group_by_news_desk,feature_news_desk,group_by_section_name,feature_section_name,group_by_type_of_material,feature_type_of_material,is_party,is_congress,is_president,is_spell,is_crossword,is_game,sentiment_tb_polar_abs,sentiment_tb_subj,sentiment_vader_pos,sentiment_vader_compound,sentiment_snow_pos,sentiment_abstract_tb_polar,sentiment_abstract_tb_polar_abs,sentiment_abstract_tb_subj,sentiment_abstract_tb_pos,sentiment_abstract_tb_neg,sentiment_abstract_vader_neu,sentiment_abstract_vader_pos,sentiment_abstract_vader_compound,sentiment_abstract_snow,sentiment_abstract_snow_abs,sentiment_abstract_snow_pos,sentiment_abstract_snow_neg
54115,1088,18,28,10,33,0,1,0.200885,0.2,0.198759,0.2,0.566265,0.6,0,0,0,0,0,0,0.150000,0.250000,0.478,0.7506,1,0.225000,0.225000,0.400000,1,0,0.680,0.173,0.2298,0.534707,0.069414,1,0
16691,1286,8,23,11,35,1,1,0.200885,0.2,0.198759,0.2,0.566265,0.6,0,0,0,0,0,0,0.211039,0.495130,0.000,0.0000,0,-0.200000,0.200000,0.400000,0,1,0.887,0.113,0.4215,0.220917,0.558166,0,1
12519,1502,22,16,8,32,0,1,0.265251,0.3,0.216640,0.3,0.566265,0.6,0,0,1,0,0,0,0.000000,0.000000,0.182,-0.3182,0,-0.250000,0.250000,0.312500,0,1,0.722,0.000,-0.6597,0.074892,0.850216,0,1
9076,1581,9,17,12,33,1,1,0.265251,0.3,0.216640,0.3,0.566265,0.6,0,0,1,0,0,0,0.000000,0.000000,0.000,-0.4939,0,-0.050000,0.050000,0.400000,0,1,0.711,0.289,0.6249,0.459658,0.080683,0,1
23387,1296,7,33,12,73,1,1,0.236273,0.2,0.229108,0.2,0.566265,0.6,0,0,0,0,0,0,0.600000,1.000000,0.000,-0.6249,0,0.383333,0.383333,0.544444,1,0,0.771,0.176,0.6808,0.032445,0.935111,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44436,300,4,10,4,35,0,1,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0,0,0.000000,0.000000,0.000,0.0000,0,0.000000,0.000000,0.000000,0,1,1.000,0.000,0.0000,0.279163,0.441675,0,1
44435,48,4,9,11,24,0,1,0.176948,0.2,0.002232,0.2,0.000000,0.0,0,0,0,0,0,0,0.000000,0.000000,0.000,-0.5267,0,0.000000,0.000000,0.000000,0,1,1.000,0.000,0.0000,0.300779,0.398442,0,1
40486,49,4,9,12,35,0,1,0.176948,0.2,0.002232,0.2,0.000000,0.0,0,0,0,0,0,0,0.078788,0.307071,0.191,0.3818,0,0.000000,0.000000,0.000000,0,1,1.000,0.000,0.0000,0.423108,0.153783,0,1
29147,47,6,9,13,20,1,1,0.176948,0.2,0.002232,0.2,0.000000,0.0,0,0,0,0,0,0,0.000000,0.000000,0.000,-0.7650,0,0.000000,0.000000,0.000000,0,1,1.000,0.000,0.0000,0.232313,0.535374,0,1


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
feature_names = X_train.columns

In [7]:
models = {'svc': SVC(random_state=42, probability=True),
          'rf': RandomForestClassifier(random_state=42)}

In [8]:
class CustomTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

# This function takes a dataframe as input and returns a scaled version
def scale_df(input_df):
    ss = StandardScaler()
    feature_names = input_df.columns
    input_df = ss.fit_transform(input_df)
    input_df = pd.DataFrame(input_df, columns=feature_names)
    return input_df

In [9]:
init_list = []
gs_list = []

# Function to run model -- input scaler and model
def run_model(mod, mod_params={}, grid_search=False):
    
    # Initial dictionary to hold model results
    results = {}
    
    pipe = Pipeline([
            ('ss', CustomTransformer(scale_df)),
            (mod, models[mod])
            ])
    
    if grid_search:
        # Instantiate list to store gridsearch results
        gs = GridSearchCV(pipe, param_grid=mod_params, cv=3, verbose=1, scoring='roc_auc', n_jobs=-1)
        gs.fit(X_train, y_train)
        pipe = gs.best_estimator_
        
    else:
        pipe.fit(X_train, y_train)
    
    # Retrieve metrics
    predictions = pipe.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(X_test)[:,1]
    y_train_pred_prob = pipe.predict_proba(X_train)[:,1]
    
    results['model'] = mod
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    results['precision'] = precision_score(y_test, predictions)
    results['specificity'] = tn / (tn + fp)
    results['recall'] = recall_score(y_test, predictions)
    results['f_score'] = f1_score(y_test, predictions)
    
    if grid_search:
        gs_list.append(results)
        print('### BEST PARAMS ###')
        display(pipe[1])
        
    else:
        init_list.append(results)
    
    print('### METRICS ###')
    display(results)
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [10]:
svc = run_model('svc')

In [13]:
rf = run_model('rf')

### METRICS ###


{'model': 'rf',
 'train_auc': 0.9999997474697802,
 'test_auc': 0.8834255867482913,
 'precision': 0.7574626865671642,
 'specificity': 0.9103345212093344,
 'recall': 0.6171776032429693,
 'f_score': 0.680161943319838}

True Negatives: 7919
False Positives: 780
False Negatives: 1511
True Positives: 2436


In [10]:
pd.DataFrame(init_list).sort_values(by='test_auc', ascending=False).reset_index(drop=True)

KeyError: 'test_auc'

In [11]:
svc_params = {
    'svc__C':[0.05, 1,0.04, 0.06, 0.045, 0.9, 0.8, 0.7, 0.6],
    'svc__gamma':[0.05, 0.1, 0.2,0.3,0.4,0.5,0.04, 0.03,0.02], 
    'svc__kernel':['rbf', 'sigmoid'],
}
svc_gs =  run_model('svc', mod_params=svc_params, grid_search=True)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


In [ ]:
svc_params = {
    'svc__C':[0.6,0.5],
    'svc__gamma':[0.02,0.01], 
    'svc__kernel':['rbf', 'sigmoid'],
}
svc_gs =  run_model('svc', mod_params=svc_params, grid_search=True)

In [15]:
rf_params = {'rf__n_estimators': [100, 200,300,400,500],
             'rf__max_depth': [10,20,30,40,50,60],
             'rf__min_samples_leaf': [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
            }
rf_gs =  run_model('rf', mod_params=rf_params, grid_search=True)

Fitting 3 folds for each of 480 candidates, totalling 1440 fits
### BEST PARAMS ###


RandomForestClassifier(max_depth=20, min_samples_leaf=2, n_estimators=500,
                       random_state=42)

### METRICS ###


{'model': 'rf',
 'train_auc': 0.9939477106721047,
 'test_auc': 0.8866342412060387,
 'precision': 0.7683776932826363,
 'specificity': 0.9159673525692609,
 'recall': 0.6143906764631366,
 'f_score': 0.6828100802477827}

True Negatives: 7968
False Positives: 731
False Negatives: 1522
True Positives: 2425


In [ ]:
gs_df = pd.DataFrame(gs_list)
gs_df.sort_values(by='test_auc', ascending=False)

In [ ]:
gs_dict = {
    svc_gs: 'SupportVectorMachineClf',
    rf_gs: 'RandomForest',
}

In [ ]:
def roc_curve_plotter(model_dict, plot_top=False):
    fig, ax = plt.subplots(1, 1, figsize=(12,10))
    axes = {}
    for i, m in enumerate(model_dict.keys()):
        axes[f'ax{i}'] = sklearn.metrics.RocCurveDisplay.from_estimator(m, X_test, y_test, ax=ax, name=model_dict[m])
    if plot_top:
        for i, a in enumerate(axes):
            if i != 0:
                axes[a].line_.set_color('gray')
                axes[a].line_.set_alpha(0.25)
                
    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--', label='Random Guess')
    plt.title('ROC-AUC Curve', fontsize=22)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.legend(fontsize=12)

In [ ]:
roc_curve_plotter(gs_dict, True)